In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import random
import shutil
# import tqdm
import glob
import numpy as np
import PIL

2022-02-09 13:29:33.529895: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


ModuleNotFoundError: No module named 'PIL'

In [2]:
!pip install pillow


  Using cached Pillow-8.4.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)


In [2]:
# physical_devices = tf.config.list_physical_devices('GPU')
# try:
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)
#     print("Memory growth set")

# except:
#   # Invalid device or cannot modify virtual devices once initialized.
    
#     pass


Memory growth set


# Load the image dataset

In [5]:
dir_path = '/mnt/D626A94C26A92E85/Practice/Dataset/PetImages/'
cat_dir = os.path.join(dir_path + 'Cat/')
dog_dir = os.path.join(dir_path + 'Dog/')
train_dir = os.path.join(dir_path + 'train/')
test_dir = os.path.join(dir_path +'test/')

path_list = [dir_path, cat_dir, dog_dir]

for path in path_list:
    print(os.path.exists(path))
    

True
True
True


In [6]:
cats_training_folder = os.path.join(dir_path + 'train/' + 'cats_training/')
dogs_training_folder = os.path.join(dir_path + 'train/' + 'dogs_training/')
cats_testing_folder = os.path.join(dir_path +'test/' + 'cats_testing/')
dogs_testing_folder = os.path.join(dir_path + 'test/' + 'dogs_testing/')
lst = [cats_training_folder, dogs_training_folder, cats_testing_folder, dogs_testing_folder]
for path in lst:
    if not os.path.exists(path):
        os.makedirs(path)
        print('made dir')
    print(os.path.exists(path))

made dir
True
made dir
True
made dir
True
made dir
True


# Data Pre-Processing


### Splitting the data as training and testing and Randomising 

In [7]:
# splitting the data based on split % 
split_p = 0.9 # 90% train data and 10 % test data
# As we have to do it for both cats and dogs it's better to write a function 
def copy_file(folder, file_names, destination):
    for name in file_names:
        file = folder + name
        dest = destination + name
        # print(file)
        # print(dest)
        shutil.copyfile(file, dest)
def split_data(folder, train_dest, test_dest, split):
    files = []
    for img in os.listdir(folder):
        # print(img)
        file = os.path.join(cat_dir + img)
        if os.path.getsize(file) > 0:
            files.append(img)
            
    train_size = int(len(files)*split)
    test_size = int(len(files) - train_size)
    print(test_size, train_size)
    if train_size+test_size == len(files):
        print("split correct")
    shuffled_data = random.sample(files, len(files))
    # print(shuffled_data)
    training_set = shuffled_data[0:train_size]
    print(len(training_set))
    testing_set = shuffled_data[train_size:]
    # print(testing_set)
    
    # Copying files to specific folder
    print(f"Before copying: {len(os.listdir(train_dest))} files")
    print(f"Before copying {len(os.listdir(train_dest))}")
    copy_file(folder, training_set, train_dest)
    copy_file(folder, testing_set, test_dest)
    print(f"After copying {len(os.listdir(train_dest))}")
    print(f"After copying {len(os.listdir(test_dest))}")
    
    
    
split_data(cat_dir, cats_training_folder, cats_testing_folder, split_p)
split_data(dog_dir, dogs_training_folder, dogs_testing_folder, split_p)

1250 11250
split correct
11250
Before copying: 0 files
Before copying 0
After copying 11250
After copying 1250
1250 11250
split correct
11250
Before copying: 0 files
Before copying 0
After copying 11250
After copying 1250


In [8]:
# finding the shape of the input image
path = glob.glob(cats_testing_folder + '*')
path[0]
img = keras.preprocessing.image.load_img(path[0])
arr = np.array(img)
print(arr.shape)

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

### Buidling the model

In [6]:
model = keras.models.Sequential([
    keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)), 
    keras.layers.MaxPool2D(2,2),
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)), 
    keras.layers.MaxPool2D(2,2), 
    keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)), 
    keras.layers.MaxPool2D(2,2),
    keras.layers.Flatten(), 
    keras.layers.Dense(512, activation = 'relu'), 
    keras.layers.Dense(1, activation = 'sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 16)        4624      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4624)              0

### Compiling the model

In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss= tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

# Preparing data flow from files 

In [8]:
datagen = keras.preprocessing.image.ImageDataGenerator()
train_data = datagen.flow_from_directory(train_dir, 
                                         batch_size = 64,
                                         class_mode = 'binary',
                                         target_size = (150, 150))
validation_data = datagen.flow_from_directory(test_dir)

Found 22498 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [9]:
model.fit(train_data, verbose = '2', epochs = 2)

Epoch 1/2


/home/venkata/anaconda3/envs/tensorflow-gpu/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  warnings.warn(
/home/venkata/anaconda3/envs/tensorflow-gpu/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  warnings.warn(
/home/venkata/anaconda3/envs/tensorflow-gpu/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  warnings.warn(
/home/venkata/anaconda3/envs/tensorflow-gpu/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  warnings.warn(
/home/venkata/anaconda3/envs/tensorflow-gpu/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corru

UnknownError: 2 root error(s) found.
  (0) Unknown:  UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7fe3ac08fc20>
Traceback (most recent call last):

  File "/home/venkata/anaconda3/envs/tensorflow-gpu/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 243, in __call__
    ret = func(*args)

  File "/home/venkata/anaconda3/envs/tensorflow-gpu/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 309, in wrapper
    return func(*args, **kwargs)

  File "/home/venkata/anaconda3/envs/tensorflow-gpu/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 785, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/venkata/anaconda3/envs/tensorflow-gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/data_adapter.py", line 801, in wrapped_generator
    for data in generator_fn():

  File "/home/venkata/anaconda3/envs/tensorflow-gpu/lib/python3.8/site-